Questions
* What should be frozen for fitting?
* Should I update variables to match combinedne results?

In [1]:
import sherpa.astro.ui as shp
import itertools

In [2]:
# configure plotting
%matplotlib inline
shp.set_ylog('data')

In [3]:
# configure fitting
shp.set_conf_opt('sigma', 1.6448536269514722) # 90% confidence interval
shp.set_stat('chi2xspecvar')
shp.set_xsabund('wilm')
shp.set_xsxsect('vern')

In [4]:
models = ['po', 'bremss']
pars = ['min', 'best', 'max']

In [5]:
def setup_po(par):
    pars = {'min': 1.43867, 'best': 1.76302, 'max': 2.02171}
    shp.set_source(shp.xstbabs.absorption*shp.xspowerlaw.chxepo)
    shp.set_par(absorption.nH, 16.4) # old value
    shp.set_par(chxepo.norm, 1.6e-4) # old value
    shp.set_par(chxepo.PhoIndex, pars[par])
    shp.freeze(absorption.nH)

def setup_bremss(par):
    pars = {'min': 33.6581, 'best': 49.756, 'max': 102.691}
    shp.set_source(shp.xstbabs.absorption*shp.xsbremss.chxebremss)
    shp.set_par(absorption.nH, 16.4) # an old value
    shp.set_par(chxebremss.norm, 1.9e-4) # old value
    shp.set_par(chxebremss.kT, pars[par], 1e-4, 200)
    shp.freeze(absorption.nH)

In [6]:
results = []
for model, par in list(itertools.product(models, pars)):
    result = [model, par]
    print('Simulating ' + model + ' ' + par)
    setup_po(par) if model is 'po' else setup_bremss(par)
    shp.fake_pha(id=1, arf=None, rmf='A_NE.rsp', exposure=5e6) # A_NE.rsp was copied from combinedne just as a sample response file
    shp.ignore(':10.0,79.0:')
    
    setup_po(par)
    shp.fit()
    pofit = shp.get_fit_results()
    result.append(pofit.rstat)
    
    setup_bremss(par)
    shp.fit()
    bremssfit = shp.get_fit_results()
    result.append(bremssfit.rstat)
    
    results.append(result)
    
    #shp.plot('fit', 'delchi')

Simulating po min
Dataset               = 1
Method                = levmar
Statistic             = chi2xspecvar
Initial fit statistic = 1823.18
Final fit statistic   = 1629.59 at function evaluation 16
Data points           = 1724
Degrees of freedom    = 1722
Probability [Q-value] = 0.944343
Reduced statistic     = 0.946336
Change in statistic   = 193.59
   chxepo.PhoIndex   1.55482     
   chxepo.norm    0.000216407 
Dataset               = 1
Method                = levmar
Statistic             = chi2xspecvar
Initial fit statistic = 10057.1
Final fit statistic   = 1550.42 at function evaluation 13
Data points           = 1724
Degrees of freedom    = 1722
Probability [Q-value] = 0.998717
Reduced statistic     = 0.900359
Change in statistic   = 8506.72
   chxebremss.kT   93.3376     
   chxebremss.norm   0.000262288 
Simulating po best
Dataset               = 1
Method                = levmar
Statistic             = chi2xspecvar
Initial fit statistic = 1507.51
Final fit statistic   = 132

In [7]:
for result in results:
    print ' '.join([str(i) for i in result])

po min 0.946336333529 0.900358962346
po best 0.768353777303 0.76201744755
po max 0.584860059814 0.596572067124
bremss min 0.842634392465 0.704078451264
bremss best 0.914053713285 0.800486622716
bremss max 0.996446504634 0.903245647131
